In [15]:
#Importing all the required libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold, GridSearchCV
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error,mean_absolute_percentage_error
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import resample

In [16]:
#read_csv to read the dataset
df = pd.read_csv('/Users/mominaali/Desktop/PhD MTSU/2nd Semester/7350/Project/SemesterProject/star_classification.csv')

df.head(10)

#seperating the classes
majority = df[df['class'].isin(['GALAXY'])]
minorityQSO = df[df['class'].isin(['QSO'])]
minorityStar = df[df['class'].isin(['STAR'])]

downSampled = resample(majority, replace=False, n_samples=len(minorityQSO)+len(minorityStar), random_state=42)

#As GALAXY class is downsampled here we are concatinating it with the other two minority classes
data_downSampled = pd.concat([downSampled, minorityQSO, minorityStar])

countUnique = data_downSampled['class'].value_counts()
print(countUnique)

data_downSampled.shape

X = downSampled.drop("class", axis=1)

X.info()

Y = downSampled['class']

Y.shape

Y.info()

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0, stratify=Y)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

sc.fit(X_train)

X_train_sc = sc.transform(X_train)
X_train_sc = pd.DataFrame(X_train_sc, columns = X_train.columns)

X_test_sc = sc.transform(X_test)
X_test_sc = pd.DataFrame(X_test_sc, columns = X_test.columns)

# label_encoder = preprocessing.LabelEncoder()

# Y_train_le = label_encoder.fit_transform(Y_train)
# Y_test_le = label_encoder.fit_transform(Y_test)

Y_train_le = Y_train
Y_test_le = Y_test

X_train_sc.shape

X_test_sc.shape

Y_train_le.shape

Y_test_le.shape




class
GALAXY    40555
STAR      21594
QSO       18961
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 40555 entries, 44131 to 13439
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   obj_ID       40555 non-null  float64
 1   alpha        40555 non-null  float64
 2   delta        40555 non-null  float64
 3   u            40555 non-null  float64
 4   g            40555 non-null  float64
 5   r            40555 non-null  float64
 6   i            40555 non-null  float64
 7   z            40555 non-null  float64
 8   run_ID       40555 non-null  int64  
 9   rerun_ID     40555 non-null  int64  
 10  cam_col      40555 non-null  int64  
 11  field_ID     40555 non-null  int64  
 12  spec_obj_ID  40555 non-null  float64
 13  redshift     40555 non-null  float64
 14  plate        40555 non-null  int64  
 15  MJD          40555 non-null  int64  
 16  fiber_ID     40555 non-null  int64  
dtypes: float6

(8111,)

In [29]:
from sklearn.decomposition import PCA


def fitness_function(params):
    n_estimators, max_depth = params
    rf_pso_new = RandomForestClassifier(n_estimators=int(round(n_estimators)), max_depth=int(round(max_depth)), random_state=42)
    afterCrossVal = cross_val_score(rf_pso_new, X_train_sc, Y_train_le, cv=5)
    avgScore = np.mean(afterCrossVal)
    return -avgScore




In [31]:
from pyswarm import pso
import numpy as np

# pip install pyswarm
bound_lower = [10, 1] 
bound_upper = [50, 5] 


bestParameter, _ = pso(fitness_function, bound_lower, bound_upper, maxiter=5)
best_rf_model = RandomForestClassifier(n_estimators=int(round(bestParameter[0])), max_depth=int(round(bestParameter[1])), random_state=42)
best_rf_model.fit(X_train_sc, Y_train_le)

y_pred_best_rf = best_rf_model.predict(X_test_sc)

Stopping search: maximum iterations reached --> 5


In [32]:
# Calculate accuracy
accuracy_rf = accuracy_score(Y_test_le, y_pred_best_rf)
print("Accuracy with Random Forest:", accuracy_rf)

Accuracy with Random Forest: 1.0
